# Predicting fluxes on grid

To do:
 
* Implement confidence intervals using `forestci` https://github.com/scikit-learn-contrib/forest-confidence-interval


In [1]:
import sys
import xarray as xr
import numpy as np
import pandas as pd
from joblib import load
from matplotlib import pyplot as plt
from datacube.utils.dask import start_local_dask
# from dask.distributed import Client,Scheduler
# from dask_jobqueue import SLURMCluster

sys.path.append('/g/data/os22/chad_tmp/NEE_modelling/')
from collect_prediction_data import collect_prediction_data

sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/')
from dea_tools.classification import predict_xr, HiddenPrints

In [2]:
# cluster = SLURMCluster(processes=2, cores=2, memory="47GB", walltime='01:00:00')
# client = Client(cluster)
# cluster.scale(cores=18)

In [3]:
client = start_local_dask(mem_safety_margin='2Gb')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 1
Total threads: 28,Total memory: 123.34 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38267,Workers: 1
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 123.34 GiB
Comm: tcp://127.0.0.1:40935,Total threads: 28
Dashboard: /proxy/43663/status,Memory: 123.34 GiB
Nanny: tcp://127.0.0.1:41035,


## Analysis Parameters

In [4]:
var = 'NEE'
results_name='NEE_2003_2021_RF.nc'
model_path = '/g/data/os22/chad_tmp/NEE_modelling/results/models/AUS_NEE_RF_model.joblib'
#data_path = '/g/data/os22/chad_tmp/NEE_modelling/results/prediction_data/prediction_data_2003_2021.nc'
t1, t2='2003','2021'

## Open model

In [5]:
model = load(model_path).set_params(n_jobs=1)

## Open predictor data

In [6]:
# data = xr.open_dataset(data_path).set_coords('spatial_ref')
data = collect_prediction_data(time_start=t1,
                             time_end=t2,
                             verbose=False,
                             export=False
                             )
data

,Array,Chunk
Bytes,12.13 GiB,2.15 MiB
Shape,"(228, 3400, 4200)","(1, 750, 750)"
Count,14041 Tasks,6840 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,12.13 GiB,2.15 MiB
Shape,"(228, 3400, 4200)","(1, 750, 750)"
Count,13861 Tasks,6840 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


### Check training and prediction variable order

In [8]:
train_vars = list(pd.read_csv('/g/data/os22/chad_tmp/NEE_modelling/results/variables.txt'))[0:-1]
train_vars=[i[:-3] for i in train_vars]

if train_vars == list(data.data_vars):
    print('All good')
else:
    raise ValueError('Variables dont match')

All good


### Predict each time-step seperately

- TO DO: fix timesteps that come back from `predict_xr`

In [ ]:
results = []

i=0
#start from 3 as these time-steps doesn't have rainfall lag values
for i in range(0, len(data.time)): 
    print(" {:03}/{:03}\r".format(i + 1, len(range(0, len(data.time)))), end="")
    with HiddenPrints():
        predicted = predict_xr(model,
                            data.isel(time=i),
                            proba=False,
                            clean=True,
                              ).compute()
    
    predicted = predicted.Predictions#.where(~mask.isel(time=i))
    predicted['time'] = data.isel(time=i).time.values
    results.append(predicted.astype('float32'))
    i+=1 

In [ ]:
ds = xr.concat([ds,ds1], dim='time').sortby('time').rename(var).astype('float32')

### Mask water and urban areas using landcover dataset

In [ ]:
mask = xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/data/1km/Landcover_1km_monthly_2002_2021.nc').isel(time=0)
mask = mask.rename({'latitude':'y', 'longitude':'x'})
mask['x'] = mask.x.astype('float32')
mask['y'] = mask.y.astype('float32')
mask['y'] = np.array([round(i,4) for i in mask.y.values])
mask['x'] = np.array([round(i,4) for i in mask.x.values])
mask = ~np.isnan(mask)

In [ ]:
ds = ds.where(mask).astype('float32')

In [ ]:
ds.to_netcdf('/g/data/os22/chad_tmp/NEE_modelling/results/predictions/'+results_name)


## Animate result for fun

In [ ]:
import xarray as xr
from IPython.display import Image
import matplotlib.pyplot as plt

import sys
sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools')
from dea_tools.plotting import xr_animation


In [ ]:
# var='NEE'
# ds = xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/results/predictions/NEE_2003_2021_LGBM.nc')

In [ ]:
path = '/g/data/os22/chad_tmp/NEE_modelling/results/figs/'+var+'_mystudy_RF.gif'

xr_animation(ds.to_dataset(),
            bands=[var],
            show_date='%b %Y',
            width_pixels=600,
            output_path=path,
            show_colorbar=True,
            colorbar_kwargs={'colors': 'black'},
            # show_gdf=poly_gdf,
            interval=200, 
            show_text=var+' gC/m2/month',
            # gdf_kwargs={'edgecolor': 'grey', 'linewidth':0.5}, 
            imshow_kwargs={'cmap': 'RdBu_r','vmin': -50, 'vmax': 50}#
            )

# Plot animation
plt.close()
Image(path, embed=True)